# <font color="#418FDE" size="6.5" uppercase>**Transforming Iterables: map, filter, zip, enumerate**</font>

>Last update: 20251207.
    
By the end of this Lecture, you will be able to:
- Use map and filter to apply functions and predicates lazily over iterables in Python 3.12. 
- Combine iterables using zip and annotate them with indices using enumerate. 
- Refactor imperative loops into expressions that leverage functional built-ins where appropriate. 


## **1. Lazy Transformations with map in Python 3.12**

### **1.1. From map Objects to Lists: Basic Creation and Conversion**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_04/Lecture_A/image_01_01.jpg?v=1765163810" width="250">



>* map defines a transformation as a recipe
>* Results are computed lazily only when iterated

>* Convert lazy map objects into concrete containers
>* list(map(...)) computes all results, storing them

>* Use lazy map for large, streaming data
>* Convert map to list for convenience, reuse



In [ ]:
#@title Python Code - From map Objects to Lists: Basic Creation and Conversion

# Demonstrate lazy map object creation and later list conversion behavior.
# Show that map does nothing until results are explicitly consumed.
# Compare iterating directly over map with converting map results into a list.

# Create a simple list of Fahrenheit temperatures for demonstration purposes.
fahrenheit_readings = [32, 50, 68, 77, 86]

# Define a function that converts Fahrenheit to Celsius and prints each conversion.
def to_celsius(fahrenheit_value):
    print("Converting value", fahrenheit_value, "from Fahrenheit to Celsius now.")
    return (fahrenheit_value - 32) * 5 / 9

# Create a map object that describes the lazy temperature conversion transformation.
lazy_celsius_map = map(to_celsius, fahrenheit_readings)

# Show that creating the map object did not immediately perform any conversions.
print("Map object created lazily, no conversions performed yet.")

# Consume the map object by converting it into a list, forcing all conversions eagerly.
celsius_list = list(lazy_celsius_map)

# Print the fully realized Celsius list, now stored eagerly in memory.
print("All conversions completed, Celsius list is:", celsius_list)



### **1.2. Combining Built-ins and Lambda Functions with map**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_04/Lecture_A/image_01_02.jpg?v=1765163860" width="250">



>* Use built-in functions with map for transformations
>* Add inline lambda steps for custom lazy processing

>* Use built-ins then lambdas for staged cleaning
>* map stays lazy, efficient for large streams

>* Chain map with built-ins and lambdas sequentially
>* Creates lazy, modular, easily adjustable data pipelines



In [ ]:
#@title Python Code - Combining Built-ins and Lambda Functions with map

# Demonstrates combining built-ins and lambda functions using map lazily.
# Shows a simple cleaning pipeline for messy temperature strings in Fahrenheit.
# Prints original values and cleaned, clamped numeric temperatures clearly.

# Create messy temperature strings with spaces, symbols, and inconsistent formatting.
raw_temps = [" 72 F", "-5F ", " 110 F", "N/A", " 32 f", " 212F"]

# First map step uses built-in str.strip to remove surrounding whitespace lazily.
stripped = map(str.strip, raw_temps)

# Second map step uses built-in str.upper to normalize letter casing lazily.
uppercased = map(str.upper, stripped)

# Third map step uses lambda to keep only entries ending with letter F.
only_fahrenheit = map(lambda text: text if text.endswith("F") else "MISSING", uppercased)

# Fourth map step uses lambda to extract numeric part or replace missing with default.
numeric_strings = map(lambda text: text[:-1] if text != "MISSING" else "68", only_fahrenheit)

# Fifth map step uses built-in float to convert numeric strings into floating temperatures.
numeric_temps = map(float, numeric_strings)

# Sixth map step uses lambda to clamp temperatures into realistic indoor range values.
clamped_temps = map(lambda t: max(32.0, min(90.0, t)), numeric_temps)

# Finally, materialize lazy pipeline into list and print original and cleaned pairs.
cleaned_list = list(clamped_temps)

for original, cleaned in zip(raw_temps, cleaned_list):
    print("Original:", original, "=>", "Cleaned Fahrenheit:", cleaned)



### **1.3. Readability Trade-offs: map vs List Comprehensions**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_04/Lecture_A/image_01_03.jpg?v=1765163924" width="250">



>* Functional map is concise for simple transformations
>* Comprehensions show complex logic and iteration more clearly

>* Inline functional conditions quickly become dense, opaque
>* Comprehensions show output, source, and conditions clearly

>* Match style to project and team conventions
>* Choose clearest form; avoid dense functional chains



In [ ]:
#@title Python Code - Readability Trade-offs: map vs List Comprehensions

# Compare map and comprehension readability for simple transformations.
# Show how a named function with map can stay very clear.
# Show when a list comprehension might read more naturally than map.

# Create a small list of raw temperature strings in Fahrenheit.
raw_temps_fahrenheit = [" 72F", "68F ", "  75F", "60F", "  80F "]

# Define a simple cleaning function that strips spaces and removes the F suffix.
def clean_temp_string(temp_text):
    cleaned_text = temp_text.strip().removesuffix("F")
    return cleaned_text

# Use map with the named function, which stays very readable here.
cleaned_with_map = list(map(clean_temp_string, raw_temps_fahrenheit))

# Do the same cleaning using a list comprehension, equally readable and direct.
cleaned_with_comprehension = [clean_temp_string(text) for text in raw_temps_fahrenheit]

# Now show a denser version using map with an inline lambda expression.
cleaned_with_lambda_map = list(map(lambda text: text.strip().removesuffix("F"), raw_temps_fahrenheit))

# Print results to compare readability and confirm identical behavior.
print("Original raw temperatures:", raw_temps_fahrenheit)
print("Cleaned using map and function:", cleaned_with_map)
print("Cleaned using list comprehension:", cleaned_with_comprehension)
print("Cleaned using map and lambda:", cleaned_with_lambda_map)



## **2. Filtering Iterables with Predicates**

### **2.1. Named Functions vs Lambdas in filter**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_04/Lecture_A/image_02_01.jpg?v=1765163977" width="250">



>* filter needs a predicate deciding kept elements
>* use named functions for complex, lambdas for simple

>* Named predicates clarify intent in complex filtering
>* Encapsulated logic improves readability, testing, and maintenance

>* Use lambdas for simple, local filter rules
>* Prefer named functions for complex, reused predicates



In [ ]:
#@title Python Code - Named Functions vs Lambdas in filter

# Demonstrate filter with named predicate function versus lambda expression.
# Show when a named function improves clarity and reuse in filtering.
# Show when a lambda keeps simple one off filtering logic concise.

# Sample list of daily temperatures in Fahrenheit degrees for one short week.
temperatures_fahrenheit = [72, 65, 90, 55, 82, 60, 95]

# Named function predicate for hot days, reused and clearly documented.
def is_hot_day(temp_fahrenheit):
    return temp_fahrenheit >= 85

# Use filter with named function to keep only clearly hot days.
hot_days_named = list(filter(is_hot_day, temperatures_fahrenheit))

# Use filter with lambda for a quick mild day check, used only once.
mild_days_lambda = list(filter(lambda temp_fahrenheit: 60 <= temp_fahrenheit <= 80, temperatures_fahrenheit))

# Print original temperatures list for context and comparison clarity.
print("All temperatures Fahrenheit:", temperatures_fahrenheit)

# Print hot days selected using the named predicate function.
print("Hot days using named function:", hot_days_named)

# Print mild days selected using the inline lambda predicate.
print("Mild days using lambda:", mild_days_lambda)



### **2.2. Truthiness Rules in filter Predicates**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_04/Lecture_A/image_02_02.jpg?v=1765164033" width="250">



>* filter predicates rely on Python truthiness rules
>* truthy results are kept, falsy results discarded

>* Truthiness filters comments and sensor readings automatically
>* Implicit truthiness can hide important zero values

>* Intentionally rely on truthiness when exclusions match
>* Use explicit booleans for precise, robust filtering



In [ ]:
#@title Python Code - Truthiness Rules in filter Predicates

# Demonstrate filter truthiness behavior with numbers and strings.
# Show how falsy values like zero and empty strings are removed.
# Compare implicit truthiness filtering with explicit boolean checks.

readings = [0, 5, -3, 0, 12, 0, 8]

comments = ["   ", "Nice work!", "", "  Thanks!  ", "OK", "   "]

nonzero_readings = list(filter(lambda value: value, readings))

nonempty_comments = list(filter(lambda text: text.strip(), comments))

def valid_reading(value):
    return value >= 0 and value <= 100

explicit_readings = list(filter(valid_reading, readings))

print("Original readings list:", readings)

print("Implicit truthiness nonzero readings:", nonzero_readings)

print("Explicit valid readings including zeros:", explicit_readings)

print("Original comments list:", comments)

print("Implicit truthiness nonempty comments:", nonempty_comments)



### **2.3. List Comprehensions vs. filter: When and Why to Choose Each**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_04/Lecture_A/image_02_03.jpg?v=1765164084" width="250">



>* Comprehensions build new lists while filtering elements
>* filter reuses predicate functions, yielding selected items lazily

>* Best when transforming and filtering in one place
>* Great for readable logic and reusable concrete lists

>* Use filter with reusable predicate functions, pipelines
>* Prefer comprehensions unless filter improves structure, performance



In [ ]:
#@title Python Code - List Comprehensions vs. filter: When and Why to Choose Each

# Compare list comprehensions and filter for simple number filtering examples.
# Show filtering only, then filtering plus transformation together in one expression.
# Highlight when list comprehensions feel clearer than filter for beginners.

# Create a simple list of daily temperatures in Fahrenheit degrees.
temperatures_fahrenheit = [55, 62, 48, 70, 85, 40, 90]

# Use filter with a named predicate to keep warm days above sixty degrees.
def is_warm_day(temp_fahrenheit):
    return temp_fahrenheit > 60

warm_days_filter = list(filter(is_warm_day, temperatures_fahrenheit))

# Use a list comprehension to express the same warm day filtering logic inline.
warm_days_comprehension = [t for t in temperatures_fahrenheit if t > 60]

print("Warm days using filter:", warm_days_filter)
print("Warm days using comprehension:", warm_days_comprehension)

# Now filter and transform together, converting warm days to Celsius degrees.
warm_celsius_comprehension = [
    round((t - 32) * 5 / 9, 1) for t in temperatures_fahrenheit if t > 60
]

print("Warm days converted to Celsius:", warm_celsius_comprehension)



## **3. Structured Iteration with zip and enumerate**

### **3.1. Managing Unequal Lengths When Zipping Iterables**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_04/Lecture_A/image_03_01.jpg?v=1765164157" width="250">



>* Real-world iterables often have unequal lengths
>* Decide length policy once; let iteration enforce

>* Stop combining when any input sequence ends
>* Loop body focuses on computation, not control flow

>* Iterate to longest sequence, filling missing placeholders
>* Simplifies loops and clarifies handling of missing data



In [ ]:
#@title Python Code - Managing Unequal Lengths When Zipping Iterables

# Demonstrate zipping unequal length lists safely with default behavior.
# Show how zip stops at shortest list automatically and cleanly.
# Compare with zip_longest which preserves extra data using explicit placeholders.

from itertools import zip_longest

products = ["A100", "B200", "C300", "D400"]
prices_dollars = [9.99, 14.50]

print("Using zip stops at shortest list only.")
for product, price in zip(products, prices_dollars):
    print("Product", product, "has price", price)

print("\nUsing zip_longest keeps all products visible.")
for product, price in zip_longest(products, prices_dollars, fillvalue="MISSING"):
    print("Product", product, "has price", price)



### **3.2. Pairing Index and Item with enumerate()**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_04/Lecture_A/image_03_02.jpg?v=1765164203" width="250">



>* Manual index tracking is verbose and fragile
>* Use index–item pairs for clearer, automatic iteration

>* enumerate tracks item positions during transformations
>* index–value pairs feed cleanly into map, filter

>* Refactor counter-based loops using index–item pairs
>* Gain clearer, pipeline-style iteration across many domains



In [ ]:
#@title Python Code - Pairing Index and Item with enumerate()

# Demonstrate pairing index and item using enumerate clearly.
# Compare manual index tracking with enumerate based iteration.
# Show filtering with enumerate while keeping original positions.

responses = ["yes", "", "no", "maybe", "", "yes"]
manual_index = 0
print("Manual index tracking for responses list:")
for response in responses:
    if response == "":
        print("Missing response at position", manual_index)
    manual_index += 1

print("\nUsing enumerate for responses list:")
for index, response in enumerate(responses):
    if response == "":
        print("Missing response at position", index)

print("\nFiltering with enumerate and list comprehension:")
missing_pairs = [(i, r) for i, r in enumerate(responses) if r == ""]
for index, response in missing_pairs:
    print("Missing response stored from position", index)



### **3.3. Layered iteration with nested zip and enumerate**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_04/Lecture_A/image_03_03.jpg?v=1765164259" width="250">



>* Use zip and enumerate for complex structures
>* Create indexed region–year pairs, avoiding nested loops

>* Use nested zip and enumerate for hierarchies
>* Each layer transforms iterables, clarifying iteration intent

>* Zip and enumerate align sensor readings over time
>* Layered iteration replaces manual indices, reducing errors



In [ ]:
#@title Python Code - Layered iteration with nested zip and enumerate

# Demonstrate layered iteration using zip and enumerate together clearly.
# Pair students with courses, then enumerate modules and lessons hierarchically.
# Show how nested enumeration replaces multiple manual index variables.

students = ["Alice", "Bob", "Cara", "Dan"]
courses = ["Physics", "History", "Biology", "Art"]
modules_per_course = [["Forces", "Waves"], ["Ancient", "Modern"], ["Cells", "Genetics"], ["Drawing", "Color"]]

# Zip students with courses and modules, then enumerate each paired course group.
for course_index, (student, course, modules) in enumerate(zip(students, courses, modules_per_course), start=1):
    print(f"Course {course_index}: {course} for {student}.")
    
    # Enumerate modules inside each course, adding a second structured index layer.
    for module_index, module in enumerate(modules, start=1):
        print(f"  Module {course_index}.{module_index}: {module}.")

print("Finished layered iteration using zip and enumerate.")



# <font color="#418FDE" size="6.5" uppercase>**Transforming Iterables: map, filter, zip, enumerate**</font>


In this lecture, you learned to:
- Use map and filter to apply functions and predicates lazily over iterables in Python 3.12. 
- Combine iterables using zip and annotate them with indices using enumerate. 
- Refactor imperative loops into expressions that leverage functional built-ins where appropriate. 

In the next Lecture (Lecture B), we will go over 'Ordering and Aggregation: sorted, min, max, sum, any, all'